In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('/Users/dbao/Downloads/research/rc_scripts/analysis')

from final_analysis.process_helper.df_choice_add_features import df_choice_add_features
from final_analysis.process_helper.df_map_add_features import df_map_add_features
from final_analysis.process_helper.df_subset_1undo_submit import df_subset_1undo_submit
from final_analysis.process_helper.df_subset_1state import df_subset_1state

from scipy.stats import spearmanr

In [3]:
%load_ext rpy2.ipython

from rpy2.robjects.packages import importr
# import R's "base" package
lme4 = importr('lme4')

/Users/dbao/opt/anaconda3/envs/road/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version. The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


# Load data

In [6]:
home_dir = '/Users/dbao/Library/CloudStorage/GoogleDrive-sherrydbao@gmail.com/.shortcut-targets-by-id/1sYZNVBbYCrHcHjo-9xdKcA4wLnob9ZEq/'+'/road_construction/data/2022_online/'
data_dir = home_dir + 'R_analysis_data/'
out_dir = home_dir + 'figures/figures_final/'
out_dir2 = home_dir + 'stat_model/final/'

In [7]:
data_choice_level = pd.read_csv(data_dir +  'choice_level/choicelevel_data.csv') # first column is index  
data_choice_level = data_choice_level.rename(columns={'Unnamed: 0':'index_copy'})
data_choice_level = df_choice_add_features(data_choice_level)
sc_data_choice_level = data_choice_level[data_choice_level['condition']==1].reset_index(drop=True) 

data_puzzle_level = pd.read_csv(data_dir +  'data.csv') # first column is subject
data_puzzle_level = df_map_add_features(data_puzzle_level, data_choice_level)
single_condition_data = data_puzzle_level[data_puzzle_level['condition']==1].copy()
single_condition_data = single_condition_data.reset_index(drop=True)

In [8]:
data_subset_before1undo = df_subset_1undo_submit(sc_data_choice_level)
state_undo_1undo = df_subset_1state(data_subset_before1undo, data_puzzle_level, single_condition_data, f"(firstUndo ==1)", 1)
state_notundo_1undo = df_subset_1state(data_subset_before1undo, data_puzzle_level, single_condition_data, f"(submit ==1)", 0)
state_df_1undo = pd.concat([state_undo_1undo,state_notundo_1undo])
state_df_1undo = state_df_1undo.reset_index(drop=True)

# only keep rows with RPE to be non-na
state_df_1undo = state_df_1undo[state_df_1undo['RPE'].notna()]

# Error detection

In [15]:
noerror = state_df_1undo.loc[state_df_1undo.cumulative_error == 0, :]
undo_noerror_avg = noerror.groupby("subjects")['firstUndo'].mean()
print(undo_noerror_avg.sem())
print(undo_noerror_avg.mean())

error = state_df_1undo.loc[state_df_1undo.cumulative_error == 1, :]
undo_error_avg = (error['firstUndo'] == 0).groupby(error["subjects"]).mean()
print(undo_error_avg.sem())
print(undo_error_avg.mean())

0.025541079466538757
0.16220896535707383
0.031222219656919854
0.6526613046286153


# Context RPE

## check the correlation between predictors

In [21]:
test = state_df_1undo.loc[state_df_1undo.cumulative_error==0,:]
# print(spearmanr(test.groupby(['subjects'])['cumulative_error'].mean() , test.groupby(['subjects'])['RPE'].mean()))
print(spearmanr(test.groupby(['subjects'])['currNumCities'].mean() , test.groupby(['subjects'])['RPE'].mean()))
print(spearmanr(state_df_1undo[state_df_1undo.error==False].groupby(['subjects'])['RT1_log'].mean() , state_df_1undo[state_df_1undo.error==False].groupby(['subjects'])['RPE'].mean()))

SpearmanrResult(correlation=0.27554470082699956, pvalue=0.0057718510472138055)
SpearmanrResult(correlation=-0.044823747680890536, pvalue=0.6595402434931723)


## undo

In [24]:
%%R -i state_df_1undo
model0 = lme4::glmer(firstUndo ~  1 + currNumCities + (1 |subjects) + (1|puzzleID),
                                  data = state_df_1undo[state_df_1undo$cumulative_error==0,], family = binomial(link="logit"))
model1 = lme4::glmer(firstUndo ~  1 + RPE + currNumCities + (1 |subjects) + (1|puzzleID),
                                  data = state_df_1undo[state_df_1undo$cumulative_error==0,], family = binomial(link="logit"))
model2 = lme4::glmer(firstUndo ~  1 + avg_numCities_before + currNumCities + (1 |subjects) + (1|puzzleID),
                                  data = state_df_1undo[state_df_1undo$cumulative_error==0,], family = binomial(link="logit"))
print(summary(model0))
print(summary(model1))
print(summary(model2))
# save(model_undo, state_df_1undo, file=file.path(out_dir2, file=paste("model_undo_rt1.RData", sep="")))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: firstUndo ~ 1 + currNumCities + (1 | subjects) + (1 | puzzleID)
   Data: state_df_1undo[state_df_1undo$cumulative_error == 0, ]

     AIC      BIC   logLik deviance df.resid 
  1044.5   1066.5   -518.2   1036.5     1821 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5462 -0.2579 -0.1149 -0.0474  6.7645 

Random effects:
 Groups   Name        Variance Std.Dev.
 subjects (Intercept) 8.6307   2.9378  
 puzzleID (Intercept) 0.9523   0.9759  
Number of obs: 1825, groups:  subjects, 99; puzzleID, 45

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)     5.0965     1.6269   3.133  0.00173 ** 
currNumCities  -1.0626     0.1944  -5.467 4.58e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
currNumCits -0.963
Generalized linear mixed model fit

In [23]:
%%R
# model comparison
anova(model0, model1)

Data: state_df_1undo[state_df_1undo$cumulative_error == 0, ]
Models:
model0: firstUndo ~ 1 + currNumCities + (1 | subjects) + (1 | puzzleID)
model1: firstUndo ~ 1 + RPE + currNumCities + (1 | subjects) + (1 | puzzleID)
       npar    AIC    BIC  logLik deviance  Chisq Df Pr(>Chisq)
model0    4 1044.5 1066.5 -518.24   1036.5                     
model1    5 1046.5 1074.0 -518.23   1036.5 0.0122  1      0.912
